# Tutorial CS001: Load Forecasting with LSTM

This tutorial demonstrates how to implement a Long Short-Term Memory (LSTM) network for short-term load forecasting. You will learn how to:
1. Preprocess energy time series data.
2. Build and train an LSTM model using PyTorch.
3. Evaluate model performance using standard metrics.
4. Visualize forecast results.

In [ ]:
import sys
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Add src to path
sys.path.append(os.path.abspath('../../src'))

from models.rnn import LSTMModel
from preprocessing.pipeline import DataPipeline
from utils.metrics import evaluate_all
from utils.plotting import plot_forecast, set_style
from utils.trainer import Trainer

set_style()

## 1. Data Preparation

We use a public electricity load dataset (Dataset A) for this tutorial. First, we download/generate the data and apply a standard preprocessing pipeline.

In [ ]:
from data.download import download_public_dataset

# Ensure project root is in path for download script
sys.path.append(os.path.abspath('../..'))
data_path = download_public_dataset('D001', save_dir='../../data/raw')
df = pd.read_csv(data_path)
df.head()

In [ ]:
pipeline = DataPipeline(method='standard')
df = pipeline.add_calendar_features(df)

# Split data temporally (no leaks!)
train_df, val_df, test_df = pipeline.split_data(df)

# Scale target
target_col = 'load'
train_scaled = pipeline.fit_transform(train_df, [target_col])
val_scaled = pipeline.transform(val_df, [target_col])
test_scaled = pipeline.transform(test_df, [target_col])

# Create windowed sequences (X, y)
WINDOW_SIZE = 24  # Look back 24 hours
X_train, y_train = pipeline.create_sequences(train_scaled, WINDOW_SIZE)
X_val, y_val = pipeline.create_sequences(val_scaled, WINDOW_SIZE)
X_test, y_test = pipeline.create_sequences(test_scaled, WINDOW_SIZE)

train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float()), batch_size=64)
test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()), batch_size=64)

## 2. Model Implementation

We use a 2-layer LSTM with 64 hidden units.

In [ ]:
input_size = 1  # only load
hidden_size = 64
num_layers = 2
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = torch.nn.MSELoss()
trainer = Trainer(model, criterion, patience=5)

# Train for a few epochs for demonstration
trainer.train(train_loader, val_loader, epochs=10, save_path='lstm_best.pth')

## 3. Evaluation

We evaluate the model on the held-out test set.

In [ ]:
y_pred_scaled = trainer.predict(test_loader)

# Inverse transform to get actual MW values
y_pred = pipeline.scaler.inverse_transform(y_pred_scaled)
y_true = pipeline.scaler.inverse_transform(y_test)

metrics = evaluate_all(y_true, y_pred)
print("Test Metrics:", metrics)

## 4. Visualization

Visualizing the results for the first 168 hours (one week) of the test set.

In [ ]:
plot_forecast(y_true[:168], y_pred[:168], title="LSTM Load Forecast (One Week)", ylabel="Load (MW)", save_path='../../figures/cs001_lstm_forecast.pdf')